# Data Preparation for Neo4j

In [14]:
import utils
import pandas as pd

We load our original JSON dataset and transform it the same way we did for the Cassandra database.

In [15]:
merged_df = utils.prepare_dataset(utils.DATASET_PATH)

## Creating the nodes

### Earthquakes

In [16]:
# We first geolocations to a format Neo4j can understand
merged_df['coordinates'] = merged_df['coordinates'].apply(lambda x: {'latitude': x[1], 'longitude': x[0], 'height': x[2]})

In [17]:
# What appears in node is its first property, so we add a description for
# each earthquake
merged_df.insert(
    loc=0,
    column='description',
    value="Mag " + merged_df['mag'].astype(str) + " in " + merged_df['place'],
)

In [18]:
earthquakes_df = pd.DataFrame(
    {
        'id': merged_df['id'],
        'type': 'node',
        'properties': merged_df.to_dict(orient='records'),
    },
)

earthquakes_df['labels'] = earthquakes_df.apply(lambda x: ['Earthquake'], axis=1)

### Networks

In [19]:
neo4j_networks = pd.DataFrame(
    {
        'type': 'node',
        'labels' : 'Network',
        'id': merged_df['net'].unique(),
    }
)

neo4j_networks['labels'] = neo4j_networks['labels'].apply(lambda x: [x])

## Creating the relationships

### `RECORDED_BY` relationship (between an earthquake and its source network)

In [20]:
relationships_df = pd.DataFrame({
    'type': 'relationship',
    'label': 'RECORDED_BY',
    'start': merged_df['id'],
    'end': merged_df['net'],
})

The ID of the earthquake is always included in `ids`, as shown by the empty
list returned by the following:

In [21]:
merged_df[merged_df.apply(lambda x: len(x['ids']) == 1 and x['id'] != x['ids'][0], axis=1)]

,description,id,mag,place,time,updated,url,detail,felt,cdi,...,ids,sources,types,nst,dmin,rms,gap,magtype,type,coordinates


### `LINKED_TO` relationship (between earthquake and other earthquakes)

We now create the `LINKED_TO` relationships. This creates a relationship between from any earthquake with an associated ID (in the `ids` property) different from its own, and the designated earthquake.

In [22]:
for index, row in merged_df[merged_df.apply(lambda x: len(x['ids']) > 1, axis=1)].iterrows():
    for id in row['ids']:
        # We verify the id is not the earthquake’s own ID
        if id != row['id']:
            # We create the relationship and add it to `relationships_df`
            relationships_df = pd.concat([
                relationships_df,
                pd.DataFrame({
                    'type': 'relationship',
                    'label': 'LINKED_TO',
                    'start': row['id'],
                    'end': id,
                }, index=range(1))
            ], ignore_index=True)

However, we notice all referenced earthquakes are not included in our original dataset.

### `ALSO_RECORDED_BY` relationship (for additional source networks)

In [23]:
for index, row in merged_df[merged_df.apply(lambda x: len(x['sources']) > 1, axis=1)].iterrows():
    for id in row['sources']:
        # We verify the id is not the earthquake’s own ID
        if id != row['net']:
            # We create the relationship and add it to `relationships_df`
            relationships_df = pd.concat([
                relationships_df,
                pd.DataFrame({
                    'type': 'relationship',
                    'label': 'ALSO_RECORDED_BY',
                    'start': row['id'],
                    'end': id,
                }, index=range(1))
            ], ignore_index=True)

### Final processing for relationships

In [24]:
# Finally, we apply the correct format for APOC, which requires `start` and
# `end` to be structured like nodes, and we also add an `id` property.
relationships_df['start'] = relationships_df['start'].apply(lambda x: {'id': x})
relationships_df['end'] = relationships_df['end'].apply(lambda x: {'id': x})
relationships_df['id'] = relationships_df.index

In [25]:
relationships_df


,type,label,start,end,id
0,relationship,RECORDED_BY,{'id': 'nc72001620'},{'id': 'nc'},0
1,relationship,RECORDED_BY,{'id': 'nc72001615'},{'id': 'nc'},1
2,relationship,RECORDED_BY,{'id': 'ak10729211'},{'id': 'ak'},2
3,relationship,RECORDED_BY,{'id': 'nc72001605'},{'id': 'nc'},3
4,relationship,RECORDED_BY,{'id': 'ak10729207'},{'id': 'ak'},4
...,...,...,...,...,...
8183,relationship,ALSO_RECORDED_BY,{'id': 'us2013puch'},{'id': 'ak'},8183
8184,relationship,ALSO_RECORDED_BY,{'id': 'us2013puct'},{'id': 'ak'},8184
8185,relationship,ALSO_RECORDED_BY,{'id': 'us2013pucx'},{'id': 'mb'},8185
8186,relationship,ALSO_RECORDED_BY,{'id': 'ak10709403'},{'id': 'us'},8186


## Saving the DataFrames

In [26]:
earthquakes_df.to_json('data/merged_df.jsonl', orient='records', lines=True)
neo4j_networks.to_json('data/merged_df.jsonl', orient='records', lines=True, mode='a')
relationships_df.to_json('data/merged_df.jsonl', orient='records', lines=True, mode='a')